### Notebook to test the Wine Contest with results from Peter Guth

Carlos H Grohmann
2022 - last update 2022-06-09

In [1]:
# imports
import sys,os
import pandas as pd
import numpy as np
from scipy.special import ndtri
import seaborn as sns

import demix_wine_functions as dw
from IPython.display import display

import qgrid

In [2]:
# set data dir -CHANGE TO YOURS HERE!!
base = '.'
date = 'files_2022_06_01'
datadir = f'{base}/csv_files/{date}'
tables_dir = f'{base}/Friedmans_tables'

# list csv files
csv_files = os.listdir(datadir)
print('Available CSV files: \n')
for f in csv_files:
    print(f)

Available CSV files: 

Elev_diff_stats.csv
Slope_diff_stats.csv
Roughness_diff_stats.csv


In [3]:
# define here a list of the files to be opened:
# selected_csv_files = csv_files # opens ALL the files
selected_csv_files = ['Elev_diff_stats.csv','Roughness_diff_stats.csv','Slope_diff_stats.csv']


# make df with one criterion per row
df_criteria = dw.make_criteria_df(selected_csv_files,datadir)

# make a list of dems, will use it later
mtrc_idx  = list(df_criteria.columns).index('METRIC')
dem_list  = list(df_criteria.columns)[mtrc_idx+1:]

AREA DEMIX_TILE REF_TYPE REF_SLOPE     METRIC       COP      SRTM  \
0     us_3dep  N38VW124H      DTM       ALL  GRID_FULL     93.85     93.85   
1     us_3dep  N38VW124H      DTM       ALL    ELD_MIN    -35.41    -34.70   
2     us_3dep  N38VW124H      DTM       ALL    ELD_Max     72.00     66.08   
3     us_3dep  N38VW124H      DTM       ALL   ELD_Mean     17.88     14.70   
4     us_3dep  N38VW124H      DTM       ALL    ELD_AVG     10.17      9.74   
...       ...        ...      ...       ...        ...       ...       ...   
4867   brazil  S24TW047G      DTM     STEEP  SMD_AB_MD      0.43      0.53   
4868   brazil  S24TW047G      DTM     STEEP  SMD_AB_MN      4.73      6.97   
4869   brazil  S24TW047G      DTM     STEEP      SMD_N  21884.00  21884.00   
4870   brazil  S24TW047G      DTM     STEEP   SMD_LE90     17.97     18.66   
4871   brazil  S24TW047G      DTM     STEEP   SLOPE_PC     17.17     17.17   

          ALOS      NASA     ASTER    FABDEM  
0        93.85     93.85     93.85       NaN  
1       -29.11    -31.70    -43.26       NaN  
2        64.17     72.08     84.97       NaN  
3        13.60     17.93     14.69       NaN  
4         8.91      9.99     11.99       NaN  
...        ...       ...       ...       ...  
4867      2.81      0.50      0.81      0.51  
4868     26.69      7.50      5.55      9.85  
4869  21954.00  21884.00  21884.00  21884.00  
4870     38.36     18.79     20.37     20.94  
4871     17.23     17.17     17.17     17.17  

[4872 rows x 11 columns]

In [4]:
# qshow(df_criteria)
grid = qgrid.QGridWidget(df=df_criteria)
display(grid)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [7]:
# get changed (or not) df from qgrid
df_for_ranking = grid.get_changed_df()

# calculate ranks for criteria (error metrics) in dataframes
df_ranks = dw.make_rank_df(df_for_ranking,dem_list)

#friedman stats
dw.friedman_stats(df_ranks,dem_list,tables_dir,cl=0.05)
print()

# DEMs ranked
dw.print_dems_ranked(df_ranks,dem_list)
print()

# apply Bonferroni-Dunn test
df_bd = dw.bonferroni_dunn_test(df_ranks,dem_list,alpha=0.95)
df_bd.style.applymap(lambda v: 'opacity: 0%;' if (v==0) else None)

n = 1638 (number of criteria)
k = 6 (number of DEMs)
cf = 120393.0
sum of ranks = 28858.0
sum of (ranks squared) = 145107834.0
sum of (squared ranks) = 116702.0
chi_r = 70571.525
For k=6, CL=0.05, and N=1638, the critical value to compare is chi_crit=11.038
Yay!! We can reject the null hipothesis and go to the Post-Hoc analysis!!

             rank_sum  rank
FABDEM_rank    3410.0   1.0
COP_rank       3814.0   2.0
ALOS_rank      4679.0   3.0
NASA_rank      4904.0   4.0
SRTM_rank      5591.0   5.0
ASTER_rank     6460.0   6.0

